# Projet IA

## Creation d'un système de recommandation en Python

### Équipe :
- BACALERIE Florent
- BERTHIER Sylvain
- GAUVIN Sarah

In [1]:
#Installation des librairies necessaire
#!pip install scikit-surprise

In [2]:
#pip install -U surprise

In [30]:
#Importation des librairies
import pandas as pd
import surprise
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split
from tqdm import tqdm
import pickle

In [4]:
#Changement du format pour la méthode describe()
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [5]:
# Load the data from a CSV file
anime_df = pd.read_csv('./datas/anime.csv')
rating_df = pd.read_csv('./datas/rating.csv')


In [6]:
print(anime_df.info(), '\n\n', anime_df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB
None 

        anime_id  rating   members
count   12294.0 12064.0   12294.0
mean    14058.2     6.5   18071.3
std     11455.3     1.0   54820.7
min         1.0     1.7       5.0
25%      3484.2     5.9     225.0
50%     10260.5     6.6    1550.0
75%     24794.5     7.2    9437.0
max     34527.0    10.0 1013917.0


In [7]:
print(rating_df.info(), '\n\n', rating_df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
None 

         user_id  anime_id    rating
count 7813737.0 7813737.0 7813737.0
mean    36728.0    8909.1       6.1
std     20997.9    8883.9       3.7
min         1.0       1.0      -1.0
25%     18974.0    1240.0       6.0
50%     36791.0    6213.0       7.0
75%     54757.0   14093.0       9.0
max     73516.0   34519.0      10.0


In [8]:
#On garde seulement les lignes où on a une note entre l'utilisateur et l'anime. 
rating_df = rating_df[rating_df['rating'].isin(range(1, 11))]
print(rating_df.info(), '\n\n', rating_df.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6337241 entries, 47 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 193.4 MB
None 

         user_id  anime_id    rating
count 6337241.0 6337241.0 6337241.0
mean    36747.9    8902.9       7.8
std     21013.4    8882.0       1.6
min         1.0       1.0       1.0
25%     18984.0    1239.0       7.0
50%     36815.0    6213.0       8.0
75%     54873.0   14075.0       9.0
max     73516.0   34475.0      10.0


On a chargé nos données provenant de deux sources : 
- anime.csv
- rating.csv

#### rating.csv 

Ce fichier contient pour chaque ligne, un utilisateur, un anime et une note. Chaque utilisateur à au moins noté 5 anime différents. Certaines lignes on une note de `-1`, cela signifie que l'utilisateur n'a pas donné de note à l'anime. 
Il est donc important d'exclure ces lignes du dataset d'entrainement. Nous passons donc de 7813737 lignes à 6337241 lignes. 1476496 lignes ont été retiré car inutiles.

#### anime.csv

ce fichier contient la description de l'ensemble des animes présent dans la base de donnée. On a plusieurs caractéritiques concerant chaque anime telque sont identifiant, le nom, les genres associés, le type, l'épisode, la notation global de l'anime, ... .


In [9]:
# Création d'un objet Reader
reader = Reader(rating_scale=(1, 10))

# Création de l'objet Dataset à partir des données et de l'objet Reader
data = Dataset.load_from_df(rating_df, reader)

#Sépération entre entrainement et test
train_set, test_set = train_test_split(data, test_size=0.2)

### reader
L'objet Reader permet de lire et interpreter les données des films
Il permet de définir le format des données d'évaluation et de les charger dans un format utilisable par les algorithmes de recommandation de la librairie.

Le paramètre `rating_scale` permet de spécifier l'échelle de notation, ici on a des notes entre 1 (plus bas) et 10 (plus haut)

In [10]:
# Initialisation de l'algorithme SVD
algo = SVD()

# Utilisation de la validation croisée pour évaluer l'algorithme
scores = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1324  1.1345  1.1329  1.1331  1.1325  1.1331  0.0007  
Fit time          64.70   68.58   68.00   67.94   68.39   67.52   1.43    
Test time         24.49   22.51   22.17   22.43   22.31   22.78   0.86    


In [11]:
# Affichage des scores de la validation croisée
print(f"Scores de la validation croisée : {scores['test_rmse'].mean()}")

Scores de la validation croisée : 1.1330818676187262


#### Recherche des hyperparamètres optimales

In [25]:
# Entraînement de l'algorithme sur l'ensemble d'entraînement complet
algo.fit(train_set)

In [26]:
# Évaluation de l'algorithme sur l'ensemble de test
predictions = algo.test(test_set)

# Calcul du RMSE
rmse = accuracy.rmse(predictions)
print(f'RMSE : {rmse}')

RMSE: 1.1327
RMSE : 1.1326855014846207


In [27]:
# Prédiction de la note que l'utilisateur 1 donnerait à l'item 10
prediction = algo.predict(1, 10)
print(prediction)

user: 1          item: 10         r_ui = None   est = 8.11   {'was_impossible': False}


In [110]:
#Enregistrement du modèle entrainé
with open('./saved_model.pkl', 'wb') as f:
    pickle.dump(algo, f)

In [98]:
#chargement du modèle entrainé
with open('./saved_model.pkl', 'rb') as f:
    algo = pickle.load(f)

In [99]:
def get_top_10_recommendations(model, user_id):
  # Prédire les notes pour tous les films
  all_predictions = []
  for i in range(1, 34528):
    all_predictions.append(model.predict(user_id, i))
    
  # Trier les prédictions par note prédite décroissante
  all_predictions.sort(key=lambda x: x.est, reverse=True)

  # Garder les 10 meilleures prédictions
  top_10_predictions = all_predictions[:10]

  # Récupérer les IDs des films recommandés
  top_10_movie_ids = [pred[1] for pred in top_10_predictions]

  return top_10_movie_ids

In [100]:
def get_names(ids):
    names = []
    for id in ids:
        names.append(anime_df.loc[anime_df['anime_id'] == id, 'name'].values[0])
    return names
        

In [109]:
# Demander à l'utilisateur de saisir son ID
user_id = int(input("Entrez votre ID d'utilisateur : "))

# Obtenir les 10 meilleures recommendations pour l'utilisateur

top_10_movie_ids = get_top_10_recommendations(algo, user_id)

print(f"Voici la liste des films recommendé pour l'utilisateur {user_id}")
for nom in get_names(top_10_movie_ids):
    print(f"- {nom}")

Entrez votre ID d'utilisateur : 2
Voici la liste des films recommendé pour l'utilisateur 2
- Mobile Suit Gundam Unicorn
- Ginga Eiyuu Densetsu
- Koe no Katachi
- Clannad: After Story
- Hunter x Hunter (2011)
- Kuroko no Basket 3rd Season
- Higurashi no Naku Koro ni
- Fullmetal Alchemist: Brotherhood
- Hajime no Ippo
- Sword Art Online
